In [ ]:
import mediapipe as mp
import pandas as pd
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib
import numpy as np

# create a PoseLandmarker object.
base_options = python.BaseOptions(model_asset_path='pose_landmarker.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
detector = vision.PoseLandmarker.create_from_options(options)

poses = ['one', 'zero', 'neut', 'plus']

dataframes = []
labels = []

for pose in poses:
    for i in range(1, 11):
        file_path = "./images/" + pose + "_" + str(i) + ".jpg"
        labels.append(pose)
        # Load the input image and detect pose landmarks from the input image.
        image = mp.Image.create_from_file(file_path)
        detection_result = detector.detect(image)
        
        pose_landmarks = detection_result.pose_landmarks
        nose_landmark = pose_landmarks[0][0]
        x_shift, y_shift, z_shift = nose_landmark.x, nose_landmark.y, nose_landmark.z
        
        data = []
        df = pd.DataFrame(data, columns=['x', 'y', 'z'])
        
        for landmarks in pose_landmarks:
            for landmark in landmarks:
                # Access the x, y, and z coordinates of each landmark
                landmark.x -= x_shift
                landmark.y -= y_shift
                landmark.z -= z_shift
        
                new_row = [landmark.x, landmark.y, landmark.z]
                df.loc[len(df)] = new_row
        
        dataframes.append(df)

# flatten each dataframe (3 columns -> 1 long vector per row)
flattened_data = [df[['x', 'y', 'z']].values.flatten() for df in dataframes]

# prepare the data (flattened_data as features, labels as target)
X = np.array(flattened_data)  # Features
y = np.array(labels)  # Labels

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# initialize the model
knn = KNeighborsClassifier(n_neighbors=3)

# fit the model on the training data
knn.fit(X_train, y_train)

# test
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

joblib.dump(knn, 'knn_model.pkl')

I0000 00:00:1737278903.370960 5756572 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M3
W0000 00:00:1737278903.478884 5757860 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1737278903.522395 5757863 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


1.0


['knn_model.pkl']